In [1]:
import requests

## Question 1

First, we define our request URL with its query parameters. Using the Requests library, we send a GET request with the given URL and query parameters. I chose option 2 (to filter by type as we make the HTTP request) because it reduces the amount of data we are requesting from mbta therefore reducing the amount of data we have to process on our end.

In [7]:
URL = "https://api-v3.mbta.com/routes"
types = "0, 1"
query_params = {'filter[type]': types}

request = requests.get(url = URL, params = query_params)

data = request.json()
for i in data['data']:
    print(i['attributes']['long_name'])

Mattapan Trolley
Green Line B
Green Line C
Green Line D
Green Line E
